--2024-10-14 21:59:13--  https://archive.ics.uci.edu/static/public/222/bank+marketing.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘bank+marketing.zip’

bank+marketing.zip      [    <=>             ] 999.85K  1.05MB/s    in 0.9s    

2024-10-14 21:59:15 (1.05 MB/s) - ‘bank+marketing.zip’ saved [1023843]



In [28]:
import pandas as pd
df = pd.read_csv('/workspaces/MLZoomCamp/03-classification/homework/data.csv', sep=';')
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y'],
      dtype='object')

In [29]:
df_hw = df_hw = df[['age', 'job', 'marital', 'education', 'balance', 'housing',
                    'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y']]
df_hw.isnull().sum()

age          0
job          0
marital      0
education    0
balance      0
housing      0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

## Question 1

In [23]:
df_hw.education.value_counts()

education
secondary    23202
tertiary     13301
primary       6851
unknown       1857
Name: count, dtype: int64

## Question 2

In [24]:
df_hw[['previous','day']].corrwith(df_hw['pdays'])

previous    0.454820
day        -0.093044
dtype: float64

In [25]:
df_hw[['campaign','pdays']].corrwith(df_hw['day'])

campaign    0.162490
pdays      -0.093044
dtype: float64

In [26]:
df_hw[['age','day']].corrwith(df_hw['balance'])

age    0.097783
day    0.004503
dtype: float64

In [31]:
df_hw['y'] = (df_hw['y'] == 'yes').astype(int)

/tmp/ipykernel_104105/2055334780.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hw['y'] = (df_hw['y'] == 'yes').astype(int)


In [33]:
from sklearn.model_selection import train_test_split
df_full_train, df_test = train_test_split(df_hw, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [34]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_full_train = df_full_train.reset_index(drop=True)
y_train = df_train.y.values
y_val = df_val.y.values
y_test = df_test.y.values
del df_train['y']
del df_val['y']
del df_test['y']

## Question 3

In [35]:
from sklearn.metrics import mutual_info_score

In [36]:
mutual_info_score(df_full_train.y, df_full_train.contact)

0.014163614885003041

In [37]:
mutual_info_score(df_full_train.y, df_full_train.education)

0.0024578282035066477

In [38]:
mutual_info_score(df_full_train.y, df_full_train.housing)

0.00980003813867901

In [39]:
mutual_info_score(df_full_train.y, df_full_train.poutcome)

0.02925655626647966

## Question 4

In [47]:
categorical = list(df_hw.columns[df_hw.dtypes == 'object'])
numerical = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

In [45]:
df_train.columns

Index(['age', 'job', 'marital', 'education', 'balance', 'housing', 'contact',
       'day', 'month', 'duration', 'campaign', 'pdays', 'previous',
       'poutcome'],
      dtype='object')

In [41]:
df_train.dtypes

age           int64
job          object
marital      object
education    object
balance       int64
housing      object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
dtype: object

In [46]:
df_train.head()

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome
0,41,management,married,tertiary,676,no,unknown,17,jun,78,1,-1,0,unknown
1,25,student,single,secondary,1747,yes,cellular,24,jul,903,1,-1,0,unknown
2,38,blue-collar,single,primary,0,yes,cellular,17,apr,775,1,-1,0,unknown
3,49,management,single,tertiary,4465,yes,cellular,20,nov,72,2,-1,0,unknown
4,38,self-employed,married,secondary,1029,yes,cellular,4,feb,206,3,223,1,other


In [48]:
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
train_dicts[1]

{'job': 'student',
 'marital': 'single',
 'education': 'secondary',
 'housing': 'yes',
 'contact': 'cellular',
 'month': 'jul',
 'poutcome': 'unknown',
 'age': 25,
 'balance': 1747,
 'day': 24,
 'duration': 903,
 'campaign': 1,
 'pdays': -1,
 'previous': 0}

In [49]:
from sklearn.feature_extraction import DictVectorizer

In [50]:
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts)
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [52]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
accuracy = (y_pred == y_val).mean()
accuracy.round(2)

0.9

In [55]:
accuracy

0.9024552090245521

## Question 5

In [53]:
data_feature_less_importance = ['age','balance','marital','previous']

In [56]:
for feature in data_feature_less_importance:
    df_small_train = df_train.drop([feature],axis=1)
    df_small_val = df_val.drop([feature],axis=1)
    train_small_dicts = df_small_train.to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_small_train = dv.fit_transform(train_small_dicts)
    val_small_dicts = df_small_val.to_dict(orient='records')
    X_small_val = dv.transform(val_dicts)
    model_small = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
    model_small.fit(X_small_train, y_train)
    y_small_pred = model_small.predict(X_small_val)
    print(f"Sin feature {feature} score = {(y_small_pred == y_val).mean()}\
           diferencia con el modelo con todas las variables = \
          {(y_pred == y_val).mean()-(y_small_pred == y_val).mean()}")

Sin feature age score = 0.9020128290201282           diferencia con el modelo con todas las variables =           0.00044238000442387015
Sin feature balance score = 0.9022340190223402           diferencia con el modelo con todas las variables =           0.00022119000221187957
Sin feature marital score = 0.9030081840300819           diferencia con el modelo con todas las variables =           -0.0005529750055297544
Sin feature previous score = 0.9019022340190224           diferencia con el modelo con todas las variables =           0.0005529750055297544


## Question 6

In [58]:
values_c = [0.01, 0.1, 1, 10, 100]
for c in values_c:
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    accuracy = (y_pred == y_val).mean()
    print(f'C:{c}; accuracy = {accuracy.round(3)}')

C:0.01; accuracy = 0.899
C:0.1; accuracy = 0.902
C:1; accuracy = 0.902
C:10; accuracy = 0.902
C:100; accuracy = 0.901
